# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [1]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional
import qpsolvers

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course/src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [50]:
# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector and covariance matrix for the multivariate normal distribution
mean = np.random.uniform(-0.01, 0.01, N)
cov = np.random.uniform(0.0001, 0.001, (N, N))
cov = np.dot(cov, cov.T)

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Create instances of ExpectedReturn and Covariance
expected_return_estimator = ExpectedReturn()
covariance_estimator = Covariance()

# Compute the vector of expected returns (mean returns) from df
q = expected_return_estimator.estimate(X=df, inplace=False)

# Compute the covariance matrix from df
P = covariance_estimator.estimate(X=df, inplace=False)

# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1      0.006904
Asset_2     -0.004733
Asset_3      0.007938
Asset_4     -0.001970
Asset_5     -0.006164
               ...   
Asset_96     0.001116
Asset_97     0.001656
Asset_98     0.009016
Asset_99     0.009179
Asset_100   -0.005571
Length: 100, dtype: float64

Covariance matrix (P):
            Asset_1   Asset_2   Asset_3   Asset_4   Asset_5   Asset_6  \
Asset_1    0.000038  0.000030  0.000028  0.000029  0.000026  0.000031   
Asset_2    0.000030  0.000036  0.000028  0.000026  0.000025  0.000031   
Asset_3    0.000028  0.000028  0.000037  0.000028  0.000027  0.000030   
Asset_4    0.000029  0.000026  0.000028  0.000035  0.000025  0.000030   
Asset_5    0.000026  0.000025  0.000027  0.000025  0.000029  0.000026   
...             ...       ...       ...       ...       ...       ...   
Asset_96   0.000033  0.000032  0.000032  0.000032  0.000028  0.000032   
Asset_97   0.000029  0.000027  0.000027  0.000027  0.000024  0.000028   
Asset_98   0.0000

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [51]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget()

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0, upper=0.2)

# Add linear constraints
# Group 1: Sum of weights of the first 30 assets <= 0.3
G1 = pd.Series([1] * 30 + [0] * 70, index=df.columns)
constraints.add_linear(g_values=G1, sense='<=', rhs=0.3, name='Group1')

# Group 2: Sum of weights of assets 31 to 60 <= 0.4
G2 = pd.Series([0] * 30 + [1] * 30 + [0] * 40, index=df.columns)
constraints.add_linear(g_values=G2, sense='<=', rhs=0.4, name='Group2')

# Group 3: Sum of weights of assets 61 to 100 <= 0.5
G3 = pd.Series([0] * 60 + [1] * 40, index=df.columns)
constraints.add_linear(g_values=G3, sense='<=', rhs=0.5, name='Group3')

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [13]:
list(USABLE_SOLVERS)

['osqp', 'highs', 'quadprog', 'daqp', 'qpalm', 'cvxopt']

In [66]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
solvers = list(USABLE_SOLVERS)

# Initialize a DataFrame to store the results
results = pd.DataFrame(index=['solution_found', 'objective', 'primal_residual', 'dual_residual', 'duality_gap', 'runtime'], columns=solvers)

# Loop over solvers, instantiate the quadratic program, solve it and store the results
# problem = qpsolvers.Problem(
#     P=P,
#     q=q.to_numpy(),
#     G=GhAb['G'],
#     h=GhAb['h'],
#     A=GhAb['A'],
#     b=GhAb['b'],
#     # lb=constraints.box['lower'].to_numpy(),
#     # ub=constraints.box['upper'].to_numpy()
#     lb=np.zeros(N),
#     ub=np.repeat(0.2, N),
# )
for solver in solvers:
    try:
        start_time = timeit.default_timer()
        # solution = qpsolvers.solve_problem(
        #     problem=problem,
        #     solver='osqp',
        #     initvals=None,
        #     verbose=False,
        # )
        qp = QuadraticProgram(
            P=P.to_numpy(),
            q=q.to_numpy() * -1,
            G = GhAb['G'],
            h = GhAb['h'],
            A = GhAb['A'],
            b = GhAb['b'],
            lb = constraints.box['lower'].to_numpy(),
            ub = constraints.box['upper'].to_numpy(),
            solver = solver,
        )
        qp.problem_data
        qp.is_feasible()
        qp.solve()
        qp.objective_value()
        solution = qp.results.get('solution')
        end_time = timeit.default_timer()

        results.loc[:, solver] = [
            solution.found, 
            # solution.obj, 
            qp.objective_value(),
            solution.primal_residual(), 
            solution.dual_residual(), 
            solution.duality_gap(), 
            end_time - start_time
        ]
    except Exception as e:
        # If solver fails, store NaNs
        results.loc[:, solver] = [False, np.nan, np.nan, np.nan, np.nan, np.nan]
        print(f"Solver {solver} failed: {e}")

Print and visualize the results

In [67]:
results

,daqp,quadprog,highs,osqp,cvxopt,qpalm
solution_found,True,True,True,True,True,True
objective,-0.008978,-0.008978,-0.008978,-0.009615,-0.008978,-0.008978
primal_residual,0.0,0.0,0.0,0.001581,0.0,0.0
dual_residual,0.0,0.0,0.0,0.000092,0.0,0.0
duality_gap,[1.1214699871878175e-10],[1.1497138419254427e-12],[1.8000000000608675e-08],[0.0006432497076593177],[3.302204057631852e-08],[1.9451196109522692e-07]
runtime,0.021796,0.16581,0.042999,0.052051,0.115616,0.056003


## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [33]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        self.objective = self.covariance.estimate(optimization_data.return_series)

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            if self.objective.ndim == 1:
                self.objective = np.diag(self.objective)
            inv_cov_matrix = np.linalg.inv(self.objective)
            ones = np.ones(inv_cov_matrix.shape[0])
            weights = inv_cov_matrix @ ones / (ones.T @ inv_cov_matrix @ ones)
            self.solution = weights
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
asset_ids = [f'Asset_{i+1}' for i in range(100)]
constraints = Constraints(ids=asset_ids)
constraints.add_budget()

# Instantiate the MinVariance class
min_variance = MinVariance(constraints=constraints, solver_name='analytical')

# Prepare the optimization data and prepare the optimization problem
optimization_data = OptimizationData(return_series=df)

# Solve the optimization problem and print the weights
min_variance.set_objective(optimization_data)
min_variance.solve()
print("Optimal Weights:")
print(min_variance.solution)

AttributeError: 'NoneType' object has no attribute 'ndim'